## Project Setup

In [3]:
# all import statements needed for the project, for example:
import pandas as pd
import sqlalchemy as db
import sqlite3
import os

In [4]:
# add constant
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

# Part 3: Understanding the Data

In [5]:
engine = db.create_engine(DATABASE_URL)

In [6]:
def write_query_to_file(query: str, query_name: str):
    # Create the directory if it does not exist
    if not os.path.exists(QUERY_DIRECTORY):
        os.makedirs(QUERY_DIRECTORY)

    # Construct the file path using the query directory and query name
    file_path = os.path.join(QUERY_DIRECTORY, f"{query_name}.sql")

    # Write the query string to the file
    with open(file_path, "w") as f:
        f.write(query)
        f.close()

## Query 1

_**TODO:** For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi?_

In [9]:
QUERY_1 = """
SELECT strftime ('%H',pickup_datetime) AS HOUR,
COUNT(strftime ('%H',pickup_datetime)) AS Trip_counts
FROM taxi_trips
GROUP BY HOUR
ORDER BY Trip_counts DESC
"""

In [10]:
engine.execute(QUERY_1).fetchall()

[('19', 12300),
 ('18', 11824),
 ('20', 11497),
 ('21', 11046),
 ('22', 10898),
 ('14', 9944),
 ('23', 9732),
 ('17', 9728),
 ('12', 9727),
 ('13', 9566),
 ('15', 9282),
 ('09', 9247),
 ('11', 9130),
 ('08', 8898),
 ('10', 8861),
 ('16', 8114),
 ('00', 7867),
 ('07', 7025),
 ('01', 5477),
 ('02', 4151),
 ('06', 4083),
 ('03', 3047),
 ('04', 2236),
 ('05', 1792)]

In [11]:
write_query_to_file(QUERY_1, "most_popular_hour_taxi.sql")

## Query 2

_**TODO:** For the same time frame, what day of the week was the most popular to take an uber?_

In [13]:
QUERY_2 = '''
SELECT strftime ('%w',pickup_datetime) AS Day,
COUNT(strftime ('%w',pickup_datetime)) AS Trip_counts
FROM uber_trips
GROUP BY Day
ORDER BY Trip_counts DESC
'''

In [14]:
engine.execute(QUERY_2).fetchall()

[('5', 30166),
 ('6', 29599),
 ('4', 29338),
 ('3', 28328),
 ('2', 27526),
 ('0', 25834),
 ('1', 24681)]

In [15]:
write_query_to_file(QUERY_2, "most_popular_day_uber.sql")

## Query 3

_**TODO:** What is the 95% percentile of distance traveled for all hired trips during July 2013?_

In [33]:
QUERY_3 = """
WITH hired_trips AS (SELECT pickup_datetime, trip_distance as distance 
FROM taxi_trips 
WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-07-31'
UNION ALL
SELECT pickup_datetime, trip_distance as distance
FROM uber_trips
WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-07-31')
          
SELECT distance
FROM hired_trips
ORDER BY distance ASC
LIMIT 1

OFFSET (SELECT COUNT(*) FROM hired_trips) * 95 / 100 - 1 ;
          """

In [34]:
engine.execute(QUERY_3).fetchall()

[(9.871179738140432,)]

In [35]:
write_query_to_file(QUERY_3, "95%_distance.sql")

## Query 4

_**TODO:** What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?_

In [40]:
QUERY_4 = """
SELECT strftime ('%Y-%m-%d',pickup_datetime) AS DAY,
AVG(trip_distance)
FROM uber_trips
WHERE pickup_datetime between '2009-01-01' AND '2009-12-31'
GROUP BY DAY
ORDER BY COUNT(strftime ('%Y-%m-%d',pickup_datetime)) DESC
LIMIT 10
"""

In [41]:
engine.execute(QUERY_4).fetchall()

[('2009-12-11', 2.9768262340720337),
 ('2009-10-23', 2.4532370160277575),
 ('2009-08-14', 3.5497316114127453),
 ('2009-04-18', 3.3308394224390288),
 ('2009-01-31', 2.7928274073095603),
 ('2009-07-09', 3.455500799594005),
 ('2009-05-08', 3.3122468204357984),
 ('2009-03-19', 3.2952160550196434),
 ('2009-05-16', 2.773624142420374),
 ('2009-07-23', 3.676997730363371)]

In [21]:
write_query_to_file(QUERY_4, "top_10_rides.sql")

## Query 5

_**TODO:** Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?_

In [46]:
QUERY_5 ="""
SELECT date(pickup_datetime) AS date, COUNT(*) AS num
FROM (SELECT pickup_datetime FROM taxi_trips
UNION ALL
SELECT pickup_datetime FROM uber_trips)
GROUP BY date
HAVING date IN (SELECT date(DATE) FROM daily_weathers WHERE DATE BETWEEN '2014-01-01' AND '2015-01-01' ORDER BY DailyAverageWindSpeed DESC LIMIT 10)
         """

In [49]:
engine.execute(QUERY_5).fetchall()

In [24]:
write_query_to_file(QUERY_5, "windest_trips.sql")

## Query 6

_**TODO:** During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?_

In [7]:
QUERY_6 = """
SELECT weathers.DATE, weathers.HOUR, weathers.HourlyPrecipitation, weathers.HourlyWindSpeed, trips.numbers
FROM
(
SELECT strftime ('%Y-%m-%d',Date) AS DATE, strftime ('%H',Date) AS HOUR, HourlyPrecipitation, HourlyWindSpeed
FROM hourly_weathers 
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-07'
GROUP BY strftime ('%Y-%m-%d',Date), strftime ('%H', Date), HourlyPrecipitation, HourlyWindSpeed
) weathers
LEFT JOIN
(
SELECT strftime ('%Y-%m-%d',pickup_datetime) AS DATE, strftime ('%H',pickup_datetime) AS HOUR, COUNT(strftime ('%H',pickup_datetime)) as numbers FROM taxi_trips
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-07'
GROUP BY strftime ('%Y-%m-%d',pickup_datetime), strftime ('%H',pickup_datetime) 
union all
SELECT strftime ('%Y-%m-%d',pickup_datetime) AS DATE, strftime ('%H',pickup_datetime) AS HOUR, COUNT(strftime ('%H',pickup_datetime)) as numbers FROM uber_trips
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-07'
GROUP BY strftime ('%Y-%m-%d',pickup_datetime), strftime ('%H',pickup_datetime) 
) trips
ON trips.DATE = weathers.DATE AND trips.HOUR = weathers.HOUR
GROUP BY weathers.DATE, weathers.HOUR, weathers.HourlyPrecipitation, weathers.HourlyWindSpeed, trips.numbers
"""

In [9]:
engine.execute(QUERY_6).fetchall()

[('2012-10-22', '00', 0.0, 7.0, 2),
 ('2012-10-22', '01', 0.0, 5.0, 1),
 ('2012-10-22', '02', 0.0, 7.0, 1),
 ('2012-10-22', '03', 0.0, 0.0, 1),
 ('2012-10-22', '03', 0.0, 0.0, 2),
 ('2012-10-22', '04', 0.0, 0.0, None),
 ('2012-10-22', '05', 0.0, 0.0, 1),
 ('2012-10-22', '06', 0.0, 5.0, 3),
 ('2012-10-22', '07', 0.0, 3.0, 5),
 ('2012-10-22', '07', 0.0, 3.0, 7),
 ('2012-10-22', '08', 0.0, 3.0, 1),
 ('2012-10-22', '09', 0.0, 5.0, 4),
 ('2012-10-22', '09', 0.0, 5.0, 5),
 ('2012-10-22', '12', 0.0, 11.0, 4),
 ('2012-10-22', '12', 0.0, 11.0, 5),
 ('2012-10-22', '14', 0.0, 7.0, 1),
 ('2012-10-22', '14', 0.0, 7.0, 4),
 ('2012-10-22', '15', 0.0, 6.0, 1),
 ('2012-10-22', '15', 0.0, 6.0, 6),
 ('2012-10-22', '16', 0.0, 3.0, 4),
 ('2012-10-22', '16', 0.0, 3.0, 6),
 ('2012-10-22', '17', 0.0, 7.0, 2),
 ('2012-10-22', '17', 0.0, 7.0, 4),
 ('2012-10-22', '18', 0.0, 5.0, 1),
 ('2012-10-22', '18', 0.0, 5.0, 8),
 ('2012-10-22', '19', 0.0, 5.0, 2),
 ('2012-10-22', '19', 0.0, 5.0, 9),
 ('2012-10-22', '20', 0

In [8]:
write_query_to_file(QUERY_6, "hurricane.sql")